# 1) Importing the datasets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')
data_imdb = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/BERT/IMDB-Dataset.csv')
import pandas as pd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data_imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,"Taut and organically gripping, Edward Dmytryk'...",positive
2,"""Ardh Satya"" is one of the finest film ever ma...",positive
3,My first exposure to the Templarios & not a go...,positive
4,One of the most significant quotes from the en...,negative


In [ ]:
data_imdb.shape

(9998, 2)

In [ ]:
data_imdb['sentiment'].value_counts()

positive    5027
negative    4971
Name: sentiment, dtype: int64

In [ ]:
# check for null values
data_imdb.isnull().sum()

# no null values in the data

review       0
sentiment    0
dtype: int64

In [ ]:
x = data_imdb['review']
y = data_imdb['sentiment']

# 2) Data Cleaning

Removeing stopwords, punctuations
and apply lemmatization

## Create a function to clean the data

In [ ]:
import string

In [ ]:
punct = string.punctuation

In [ ]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
stopwords = list(STOP_WORDS) # list of stopwords

In [ ]:
# creating a function for data cleaning
def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
 
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens
  # if root form of that word is not pronoun then it is going to convert that into lower form
# and if that word is a proper noun, then we are directly taking lower form, because there is no lemma for proper noun

In [ ]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")
# stopwords and punctuations removed

['hello', 'beautiful', 'day', 'outside']

## Vectorization Feature Engineering (TF-IDF)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)
# tokenizer=text_data_cleaning, tokenization will be done according to this function

In [ ]:
classifier = LinearSVC()

# 3) Train the model

## Splitting the dataset into the Train and Test set

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
x_train.shape, x_test.shape
# 2198 samples in training dataset and 550 in test dataset

((7998,), (2000,))

In [ ]:
x_train.head()

4755    Kristine Watts (Molie Weeks) is broken apart, ...
2994    Three sergeants in the British army stationed ...
5867    Historically accurate? Hmm... Perhaps... if yo...
356     I was truly looking forward to this title. It ...
49      The few scenes that actually attempt a depicti...
Name: review, dtype: object

## Fit the x_train and y_train

In [ ]:
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])
# it will first do vectorization and then it will do classification

In [ ]:
clf.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7f9a7d1020e0>)),
                ('clf', LinearSVC())])

# 4) Predict the Test set results

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
# confusion_matrix
confusion_matrix(y_test, y_pred)

array([[533, 441],
       [518, 508]])

In [ ]:
# classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.51      0.55      0.53       974
    positive       0.54      0.50      0.51      1026

    accuracy                           0.52      2000
   macro avg       0.52      0.52      0.52      2000
weighted avg       0.52      0.52      0.52      2000



In [ ]:
accuracy_score(y_test, y_pred)

0.5205

In [ ]:
clf.predict(["Wow, this is a great movie"])

array(['positive'], dtype=object)

In [ ]:
clf.predict(["It's hard to enjoy these things!"])

array(['negative'], dtype=object)